In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json
from webdriver_manager.chrome import ChromeDriverManager
from pandas.io.json import json_normalize
import pandas as pd

In [78]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.whoscored.com/Matches/1376186/Live/England-Premier-League-2019-2020-Aston-Villa-Arsenal')
data = driver.execute_script("return matchCentreData;")

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
       
driver.close()

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/ollinj01/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [79]:
home_team, away_team = data["home"]["name"], data["away"]["name"]
with open(f"{home_team}_{away_team}.json", "w") as f: ## save as Arsenal_Liverpool.json in present directory
    json.dump(data, f, indent=4)

In [640]:
# data['events']

In [641]:
# df = json_normalize(data["events"], sep="_")

In [642]:
# df.isGoal.value_counts()

In [643]:
# df['Assist'] = df['isGoal'].shift(-1)

In [644]:
# mask1 = df['Assist'] == True
# df.loc[mask1]

In [645]:
# mask = (df['playerId'] == 239872) & (df['Assist'] == True)
# pepe = df.loc[mask]
# pepe

In [11]:
with open('PSG_Lille.json') as json_file:
    data = json.load(json_file)

event = []
minute = []
second = []
player = []
team = []
outcome = []
x = []
y = []
endX = []
endY = []
passtype = []


for each in data['events']:
    event.append(each['type']['displayName'])
    try:
        minute.append(each['minute'])
    except:
        minute.append('')
    try:
        second.append(each['second'])
    except:
        second.append('')
    try:
        player.append(each['playerId'])
    except:
        player.append('')
    try:
        team.append(each['teamId'])
    except:
        team.append('')
    try:
        endX.append(each['endX'])
        endY.append(each['endY'])
    except:
        endX.append('')
        endY.append('')
    outcome.append(each['outcomeType']['displayName'])
    x.append(each['x'])
    y.append(each['y'])

    for key, value in each.items():
        passType = 'OpenPlay'
        if key == 'qualifiers':
            for item in value:
                if item['type']['displayName'] == 'BigChanceCreated':
                    passType = item['type']['displayName']
                if item['type']['displayName'] == 'ShotAssist':
                    passType = item['type']['displayName']
                elif item['type']['displayName'] == 'CornerTaken':
                    passType = item['type']['displayName']
                elif item['type']['displayName'] == 'ThrowIn':
                    passType = item['type']['displayName']
            passtype.append(passType)
                
events = pd.DataFrame({'Event':event, 'Minute':minute, 'Second':second, 'Player':player, 'Team':team,
                       'Outcome':outcome, 'X':x, 'Y':y,'endX':endX, 'endY':endY, 'passtype':passtype})

In [12]:
data['events']

[{'eventId': 2,
  'expandedMinute': 0,
  'id': 2135029563,
  'isTouch': False,
  'minute': 0,
  'outcomeType': {'displayName': 'Successful', 'value': 1},
  'period': {'displayName': 'FirstHalf', 'value': 1},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'second': 0,
  'teamId': 607,
  'type': {'displayName': 'Start', 'value': 32},
  'x': 0,
  'y': 0},
 {'eventId': 2,
  'expandedMinute': 0,
  'id': 1237076355,
  'isTouch': False,
  'minute': 0,
  'outcomeType': {'displayName': 'Successful', 'value': 1},
  'period': {'displayName': 'FirstHalf', 'value': 1},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'second': 0,
  'teamId': 304,
  'type': {'displayName': 'Start', 'value': 32},
  'x': 0,
  'y': 0},
 {'endX': 51.4,
  'endY': 53.7,
  'eventId': 3,
  'expandedMinute': 0,
  'id': 1853355094,
  'isTouch': True,
  'minute': 0,
  'outcomeType': {'displayName': 'Successful', 'value': 1},
  'period': {'displayName': 'FirstHalf', 'value': 1},
  'playerId': 50835,
  'qualifiers': [{'t

In [647]:
events.head()

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype
0,Start,0,0,,607,Successful,0.0,0.0,,,OpenPlay
1,Start,0,0,,245,Successful,0.0,0.0,,,OpenPlay
2,Pass,0,1,23054,607,Successful,50.1,50.3,36.7,50.1,OpenPlay
3,Pass,0,3,117776,607,Successful,34.6,39.7,35.4,8.3,OpenPlay
4,Pass,0,5,136481,607,Successful,37.0,8.5,33.2,25.8,OpenPlay


In [648]:
mask = (events['Player'] == 239872) & (events['passtype'] == 'BigChanceCreated')
pepe = events.loc[mask]
pepe

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype


In [649]:
mask = (events['Event'] == 'Goal')
goal = events.loc[mask]
goal

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype
930,Goal,50,20,23054,607,Successful,89.2,39.2,,,OpenPlay
1163,Goal,63,20,241019,607,Successful,89.3,56.3,,,OpenPlay
1235,Goal,69,13,136481,607,Successful,90.5,34.2,,,OpenPlay
1373,Goal,79,39,239872,607,Successful,97.2,47.0,,,OpenPlay
1542,Goal,91,56,69742,607,Successful,81.9,52.8,,,OpenPlay


In [650]:
events['Assist'] = (events['Event'] == 'Goal').shift(-1)
# events['Assist_2nd'] = (events['Event'] == 'Goal').shift(-2)

In [651]:
mask = (events['Player'] == 239872) & (events['Event'] == 'Goal')
pepe_goal = events.loc[mask]
pepe_goal.head()

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype,Assist
1373,Goal,79,39,239872,607,Successful,97.2,47.0,,,OpenPlay,False


In [652]:
mask = (events['Player'] == 239872) & (events['Assist'] == True)
pepe_assist = events.loc[mask]
pepe_assist.head()

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype,Assist


In [653]:
# mask = (events['Player'] == 239872) & (events['Assist_2nd'] == True)
# pepe_assist = events.loc[mask]
# pepe_assist.head()

In [654]:
events.to_csv('Lille_Nimes.csv', index=False)

In [655]:
import os
import glob
import pandas as pd

In [656]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "pepe_all.csv", index=False, encoding='utf-8-sig')